In [57]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

TRADUCTOR_SIGFE_SIGCOM = pd.read_excel('input\\asociacion_sigfe_sigcom.xlsx')
TRADUCTOR_SIGFE_SIGCOM['Cod. SIGFE limpio'] = TRADUCTOR_SIGFE_SIGCOM['Cod. SIGFE limpio'].str.replace('.', '', regex = False)
TRADUCTOR_SIGFE_SIGCOM['Cod. SIGCOM'] = TRADUCTOR_SIGFE_SIGCOM['Cod. SIGCOM'].astype(int)
TRADUCTOR_SIGFE_SIGCOM = TRADUCTOR_SIGFE_SIGCOM.set_index('Cod. SIGFE limpio')

EXCEPCIONES_SIGFE = {
    '221299901601': 'FERNANDO BARAONA EN RRHH. ANEDIN EN GASTO GENERAL',
    '221299901602': 'Todo en RRHH',
    '221299900902': 'Cardiologia y Cardiocirugia en RRHH. UC Christus en gastos generales',
    '221299900201': 'M Meneses cargado en RRHH',
    '221299900202': 'J Andueza y Cardiocirugia en RRHH'
}

In [58]:
df = pd.read_excel('input\\SA_EstadoEjecucionPresupuestaria.xls', header = 6)
df['Item Presupuestario'] = df['Concepto Presupuestario'].str.split() \
                                                         .str[0]
df = df.set_index('Item Presupuestario')


In [93]:
columnas_utiles = ['Devengado']
agrupar_por = ['Cod. SIGCOM', 'Item en SIGCOM', 'Item Presupuestario']

unidas = pd.merge(df, TRADUCTOR_SIGFE_SIGCOM, how = 'left', left_index = True, right_index = True)
agrupados_por_sigcom = unidas.groupby(agrupar_por, dropna = False)[columnas_utiles].sum()
conteo_articulos = unidas.groupby(agrupar_por, dropna = False)[columnas_utiles].count()

In [102]:
df_devengos = pd.read_excel('input\\SA_DisponibilidadDevengoPresupuestario.xls', header = 5)
df_devengos = df_devengos[['Titulo', 'Principal', 'Concepto Presupuestario', 'Monto Vigente', 'Monto Disponible', 'Monto Consumido']]
df_devengos['Item Presupuestario'] = df_devengos['Concepto Presupuestario'].str.split() \
                                                                           .str[0]

for concepto_pres_excepcion in EXCEPCIONES_SIGFE.keys():
    query_excepcion = df_devengos.query('`Item Presupuestario` == @concepto_pres_excepcion')

    if concepto_pres_excepcion == '221299901601':
        # FER BARAONA EN RRHH, todo lo demas en gastos generales
        mask_a_rrhh = query_excepcion['Principal'].str.contains('BARAONA')
    
    elif concepto_pres_excepcion == '221299901602':
        # Todo en RRHH
        mask_a_rrhh = query_excepcion['Principal'].notna()
    
    elif concepto_pres_excepcion == '221299900902':
        # CARDIOCIRUGUA Y CARDIOLOGIA EN RRHH, LO DEMAS EN GASTOS GENERALES.
        mask_a_rrhh = (query_excepcion['Principal'].str.contains('CARDIOLOGIA')) | \
                      (query_excepcion['Principal'].str.contains('CARDIOCIRUGIA'))

    elif concepto_pres_excepcion == '221299900201':
        # M MENESES EN RRHH, LO DEMAS EN GASTOS GENERALES
        mask_a_rrhh = query_excepcion['Principal'].str.contains('MANUEL MENESES')
        
    elif concepto_pres_excepcion == '221299900202':
        # ANDUEZA Y CARDIOCIRUGIA EN RRHH, LO DEMAS EN GASTOS GENERALES.
        mask_a_rrhh = query_excepcion['Principal'].str.contains('ANDUEZA') | \
                      query_excepcion['Principal'].str.contains('CARDIOCIRUGIA')
    
    df_a_rrhh = query_excepcion[mask_a_rrhh]
    df_a_gg = query_excepcion[~mask_a_rrhh]

    print(f'Excepcion {concepto_pres_excepcion}: {EXCEPCIONES_SIGFE[concepto_pres_excepcion]}')
    with pd.ExcelWriter(f'Excepcion {concepto_pres_excepcion}_{EXCEPCIONES_SIGFE[concepto_pres_excepcion]}.xlsx') as writer:
        df_a_gg.to_excel(writer, sheet_name = 'a_gg', index = False)
        df_a_rrhh.to_excel(writer, sheet_name = 'a_rrhh', index = False)

Excepcion 221299901601: FERNANDO BARAONA EN RRHH. ANEDIN EN GASTO GENERAL
Excepcion 221299901602: Todo en RRHH
Excepcion 221299900902: Cardiologia y Cardiocirugia en RRHH. UC Christus en gastos generales
Excepcion 221299900201: M Meneses cargado en RRHH
Excepcion 221299900202: J Andueza y Cardiocirugia en RRHH


In [92]:
agrupados_por_sigcom

Devengado
Cod. SIGCOM Item en SIGCOM                                       Item Presupuestario            
8.0         EQUIPOS MENORES                                      2204013                  918438
9.0         GASES MEDICINALES                                    2204003001             12501073
16.0        MATERIAL DE OSTEOSÍNTESIS Y PRÓTESIS                 2204004004            196461778
18.0        MATERIAL MEDICO QUIRURGICO                           2204005               606865914
24.0        MATERIALES DE OFICINA, PRODUCTOS DE PAPEL E IMPRESOS 2204001                 1329242
...                                                                                          ...
NaN         NaN                                                  2905001                       0
                                                                 2905001001                    0
                                                                 2905002                 3125892
                                                                 2905002001              3125892
                                                                 Total                3793836607

[244 rows x 1 columns]